In [1]:
from __future__ import annotations

import json
import cProfile
import pstats
from asyncio import sleep

In [2]:
from py_research.db import (
    DataSource,
    RecMap,
    RefMap,
    SubMap,
    SelIndex,
    DataBase
)

In [3]:
from test_db_schema import Search, Project, Task, User, Membership, Organization

In [4]:
async def dummy_loader(name: str) -> dict:
    await sleep(0.2)
    return {
        'name': name,
        'age': 20
    }

In [5]:
data_source = DataSource(
    target=Search,
    push={
        "resultCount": Search.result_count,
        "search": Search.term,
        "results": RefMap(
            ref=Search.results,
            push={
                "project_name": Project.name,
                "project_start": Project.start,
                "project_end": Project.end,
                "project_status": Project.status,
                "tasks": RefMap(
                    ref=Project.tasks,
                    push={
                        "task_name": Task.name,
                        "task_assignees": RefMap(
                            ref=Task.assignees,
                            push=User.name,
                            match_by=User.name,
                        ),
                        "task_status": Task.status,
                    },
                ),
                "members": RefMap(
                    ref=Project.members,
                    loader=dummy_loader,
                    push={User.name, User.age},
                    rel=RecMap(
                        push={
                            Membership.role,
                        },
                    ),
                ),
            },
            pull={
                Project.org: SubMap(
                    push={
                        "organization_name": Organization.name,
                        "organization_address": Organization.address,
                        "organization_city": Organization.city,
                    },
                ),
                Project.number: SelIndex()
            },
        ),
    },
)

In [6]:
with open("./nested_data.json") as f:
    data = json.load(f)

In [7]:
Task.assignees.ctx


Table(
    _db=DataBase(
        _db=...,
        _ctx=None,
        _name=None,
        _typehint=None,
        _typevar_map={},
        _filters=[],
        _tuple_selection=None,
        backend=<py_research.db.databases.Symbolic object at 0x0000021195062AE0>,
        url=None,
        types=None,
        schema=None,
        write_to_overlay=None,
        overlay_type='name_prefix',
        validate_on_init=False,
        remove_cross_fks=False,
        _def_types={},
        _subs={},
        _metadata=MetaData(),
        _valid_caches={},
        _instance_map={}
    ),
    _ctx=None,
    _name=None,
    _typehint=py_research.db.databases.Table[test_db_schema.Task, None, py_research.db.databases.CRUD, py_research.db.databases.BaseIdx, None, None],
    _typevar_map={},
    _filters=[],
    _tuple_selection=None,
    index_by=None,
    default=False
)

In [8]:
[(v.name, v._ctx) for v in Search._values.values()]


[
    ('term', Ctx(record_type=<class 'test_db_schema.Search'>)),
    ('result_count', Ctx(record_type=<class 'test_db_schema.Search'>))
]

In [9]:
with cProfile.Profile() as pr:
  db = DataBase()
  rec = await data_source.load([data], db=db)
  
  stats = pstats.Stats(pr)

In [16]:
[s._to_dict() for s in db[Search]]

[{'term': 'test', 'result_count': 3}]

In [11]:
stats.sort_stats("cumulative")
stats.print_stats("py_research")

         104670 function calls (99062 primitive calls) in 0.285 seconds

   Ordered by: cumulative time
   List reduced from 2092 to 127 due to restriction <'py_research'>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000    0.128    0.064 C:\Users\lworm\Repos\py-research\src\py_research\db\databases.py:4310(_update_dict)
        2    0.000    0.000    0.065    0.033 C:\Users\lworm\Repos\py-research\src\py_research\db\databases.py:791(to_df)
    40/25    0.000    0.000    0.058    0.002 C:\Users\lworm\Repos\py-research\src\py_research\db\databases.py:1647(__get__)
        1    0.000    0.000    0.034    0.034 C:\Users\lworm\Repos\py-research\src\py_research\db\databases.py:3483(engine)
        3    0.000    0.000    0.011    0.004 C:\Users\lworm\Repos\py-research\src\py_research\db\datasources.py:688(_load_records)
        3    0.000    0.000    0.011    0.004 C:\Users\lworm\Repos\py-research\src\py_research\caching.py:364(__call__)
  